# 0. 1초마다 입력

In [ ]:
!pip3 install nengo
!pip3 install matplotlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
 
import nengo
from nengo.dists import Choice
from nengo.utils.ensemble import tuning_curves
from nengo.utils.matplotlib import rasterplot

In [ ]:
def input_one_hot(t):
    if(t % 1 == 0):
        return 1
    else:
        return 0

In [ ]:
model = nengo.Network()

with model:
    input_node = nengo.Node(output=input_one_hot)
    pre_neuron = nengo.Ensemble(1, dimensions=1, max_rates = [100] * 1, intercepts=[0] * 1, radius=10)
    pre_neuron.encoders = Choice([[1]])
    
    nengo.Connection(input_node, pre_neuron)
 
    post_neuron = nengo.Ensemble(1, dimensions=1, max_rates = [90] * 1, intercepts=[0] * 1)
    post_neuron.encoders = Choice([[1]])
        
    nengo.Connection(pre_neuron, post_neuron, transform=1, synapse=0.02)
    
    input_p = nengo.Probe(input_node)
    pre_p = nengo.Probe(pre_neuron.neurons, 'voltage')
    post_p = nengo.Probe(post_neuron.neurons, 'voltage')
    pre_spike_p = nengo.Probe(pre_neuron.neurons)
    post_spike_p = nengo.Probe(post_neuron.neurons)

In [ ]:
with nengo.Simulator(model) as sim: # this is the only line that changes
    plt.figure()
    plt.plot(*tuning_curves(pre_neuron, sim))
    plt.plot(*tuning_curves(post_neuron, sim))
    plt.xlabel("input value")
    plt.ylabel("firing rate")
    plt.xlim(-1, 1)
    plt.title(str(nengo.LIF()))
    sim.run(5.0)

In [ ]:
with nengo.Simulator(model) as sim: # this is the only line that changes
    plt.figure()
    plt.plot(*tuning_curves(pre_neuron, sim))
    plt.plot(*tuning_curves(post_neuron, sim))
    plt.xlabel("input value")
    plt.ylabel("firing rate")
    plt.title(str(nengo.LIF()))
    sim.run(5.0)

In [ ]:
with nengo.Simulator(model) as sim:
    sim.run(10.0)

In [ ]:
plt.figure(figsize=(12, 10))

plt.subplot(3, 1, 1)
plt.plot(sim.trange(), sim.data[input_p])
plt.ylim(0, 2)
plt.xlim(0, 10)

plt.subplot(3, 1, 2)
rasterplot(sim.trange(), sim.data[pre_spike_p])
plt.xlim(0, 10)

plt.subplot(3, 1, 3)
rasterplot(sim.trange(), sim.data[post_spike_p])
plt.xlim(0, 10)

In [ ]:
plt.figure(figsize=(12, 10))

plt.subplot(3, 1, 1)
plt.plot(sim.trange(), sim.data[input_p])
plt.xlim(0, 10)

plt.subplot(3, 1, 2)
plt.plot(sim.trange(), sim.data[pre_p])
plt.xlim(0, 10)

plt.subplot(3, 1, 3)
plt.plot(sim.trange(), sim.data[post_p])
plt.xlim(0, 10)

# 1. 음성 데이터 입력

In [ ]:
pip install numba==0.50.0 librosa==0.5.1

In [ ]:
import librosa
import librosa.display
import IPython.display
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
  
%matplotlib inline
 
audio_path = '4_3273.wav'
 
y, sr = librosa.load(audio_path)
 
plt.figure()
plt.plot(y)

In [ ]:
time_duration = int(librosa.get_duration(y=y, sr=sr) * 1000)
data_length = y.shape[0]
print("time duration: " + str(time_duration) + " ms")
print("sampling rate: " + str(sr))
print("data length: " + str(data_length))

In [ ]:
def voice_input(t):
    ms = t * 1000
    voice = y[int((data_length / time_duration) * int(ms))]
    return voice

In [ ]:
model = nengo.Network()

with model:
    voice = nengo.Node(voice_input)
    A = nengo.Ensemble(80, dimensions=1)
    nengo.Connection(voice, A, synapse=0.01)
    
    p = nengo.Probe(A, synapse=0.01)
    inp_p = nengo.Probe(voice)
    spike_p = nengo.Probe(A.neurons)
    voltage_p = nengo.Probe(A.neurons, 'voltage')

In [ ]:
with nengo.Simulator(model) as sim:
    sim.run(int(time_duration-1) / 1000)

In [ ]:
plt.figure()
plt.plot(sim.trange(), sim.data[inp_p], 'r', label="Input")
plt.plot(sim.trange(), sim.data[p], label="A output")
plt.xlim(0, 5)
plt.legend()
 
plt.figure()
rasterplot(sim.trange(), sim.data[spike_p])
plt.xlim(0, 5);

# 2. Image 입력

In [ ]:
%matplotlib inline
 
import gzip
import pickle
from urllib.request import urlretrieve
import zipfile
 
import nengo
import numpy as np
import matplotlib.pyplot as plt
from nengo.dists import Choice
from nengo.utils.ensemble import tuning_curves
from nengo.utils.matplotlib import rasterplot
 
urlretrieve("http://deeplearning.net/data/mnist/mnist.pkl.gz", "mnist.pkl.gz")
with gzip.open("mnist.pkl.gz") as f:
    train_data, _, test_data = pickle.load(f, encoding="latin1")
train_data = list(train_data)
test_data = list(test_data)

In [ ]:
data = train_data
print(data[0].shape)#  784개의 값을 가지는 array 50000개
print(data[0].shape[0])

In [ ]:
one_hot = np.zeros((data[0].shape[0], 10)) # one_hot.shpae = (50000, 10) --> 10개의 데이터 값을 가지는 array 50000개가 0으로 초기화

In [ ]:
for data in (train_data, test_data):
    one_hot = np.zeros((data[0].shape[0], 10))
    one_hot[np.arange(data[0].shape[0]), data[1]] = 1
    data[1] = one_hot
 
for i in range(3):
    plt.figure()
    plt.imshow(np.reshape(train_data[0][i], (28, 28)))
    plt.axis('off')
    plt.title(str(np.argmax(train_data[1][i])));

In [ ]:
cnt = 0

In [ ]:
def image_input(t):
    global cnt
    ms = t * 1000
    img = train_data[0][cnt]
    cnt += 1
    return img

In [ ]:
model = nengo.Network()

with model:
    input_node = nengo.Node(image_input)
    
    pre_neuron = nengo.Ensemble(784, dimensions=1)
    pre_neuron.encoders = Choice([[1]])
    
    nengo.Connection(input_node, pre_neuron)
 
    post_neuron = nengo.Ensemble(784, dimensions=1)
    post_neuron.encoders = Choice([[1]])
        
    nengo.Connection(pre_neuron, post_neuron, transform=1, synapse=0.02)
    
    input_p = nengo.Probe(input_node)
    pre_p = nengo.Probe(pre_neuron.neurons, 'voltage')
    post_p = nengo.Probe(post_neuron.neurons, 'voltage')
    pre_spike_p = nengo.Probe(pre_neuron.neurons)
    post_spike_p = nengo.Probe(post_neuron.neurons)

In [ ]:
with nengo.Simulator(model) as sim:
    sim.run(10)

In [ ]:
plt.figure(figsize=(12, 10))

plt.subplot(2, 1, 1)
rasterplot(sim.trange(), sim.data[pre_spike_p])
plt.xlim(0, 10)

plt.subplot(2, 1, 2)
rasterplot(sim.trange(), sim.data[post_spike_p])
plt.xlim(0, 10)